In [28]:
import numpy as np
from tqdm import tqdm

# Read the table data from the file
table_data = []
with open("table data.txt", "r") as file:
    lines = file.readlines()[1:]  # Skip the header line
    for line in lines:
        layer_data = line.strip().split()
        layer_number = int(layer_data[1])
        layer_type = layer_data[2]
        sdc_probability = float(layer_data[3])
        data_size = int(layer_data[4])
        total_extra_time = float(layer_data[5])
        table_data.append((layer_number, layer_type, sdc_probability, data_size, total_extra_time))

# Normalize the factors
max_sdc_probability = max(data[2] for data in table_data)
max_data_size = max(data[3] for data in table_data)
min_total_extra_time = min(data[4] for data in table_data)
max_total_extra_time = max(data[4] for data in table_data)

normalized_data = []
for layer_number, layer_type, sdc_probability, data_size, total_extra_time in table_data:
    normalized_sdc_probability = sdc_probability / max_sdc_probability
    normalized_data_size = data_size / max_data_size
    normalized_total_extra_time = (total_extra_time - min_total_extra_time) / (max_total_extra_time - min_total_extra_time) * 2 - 1
    
    normalized_data.append((layer_number, layer_type, normalized_sdc_probability, normalized_data_size, normalized_total_extra_time))

# Define the layer type weights
layer_type_weights = {
    "Conv2d": 0.9,
    "Linear": 0.8,
    "BatchNorm2d": 0.7,
    "ReLU": 0.3,
    "MaxPool2d": 0.1,
    "Dropout": 0.0
}

# Define the objective function for parameter optimization
def objective_function(params):
    w1, w2, w3, w4 = params
    alpha = 0.5
    beta = 0.5
    total_costs = []
    for layer_number, layer_type, sdc_probability, data_size, total_extra_time in normalized_data:
        layer_type_weight = layer_type_weights[layer_type]
        total_cost = w1 * layer_type_weight + w2 * sdc_probability + w3 * data_size + w4 * total_extra_time
        total_costs.append((layer_number, total_cost))

    total_costs.sort(key=lambda x: x[1], reverse=True)
    rho = 26 / 60
    sgx_layers = [layer[0] for layer in total_costs[:int(rho * len(total_costs))]]
    
    sdc_total = sum(data[2] for data in table_data if data[0] not in sgx_layers)
    et_total = sum(data[4] for data in table_data if data[0] not in sgx_layers)
    
    objective = alpha * sdc_total + beta * et_total
    
    return objective

weight_values = np.linspace(0, 1, 30)

best_params = None
best_objective = float("inf")

total_iterations = len(weight_values) ** 4

with tqdm(total=total_iterations, desc="Grid Search Progress") as pbar:
    for w1 in weight_values:
        for w2 in weight_values:
            for w3 in weight_values:
                for w4 in weight_values:
                    params = (w1, w2, w3, w4)
                    objective = objective_function(params)
                    if objective < best_objective:
                        best_params = params
                        best_objective = objective
                    pbar.update(1)

w1, w2, w3, w4 = best_params
total_costs = []
for layer_number, layer_type, sdc_probability, data_size, total_extra_time in normalized_data:
    layer_type_weight = layer_type_weights[layer_type]
    total_cost = w1 * layer_type_weight + w2 * sdc_probability + w3 * data_size + w4 * total_extra_time
    total_costs.append((layer_number, total_cost))

total_costs.sort(key=lambda x: x[1], reverse=True)
rho = 26 / 60
sgx_layers = [layer[0] for layer in total_costs[:int(rho * len(total_costs))]]
gpu_layers = [layer[0] for layer in total_costs[int(rho * len(total_costs)):]]

sgx_layers.sort()
gpu_layers.sort()

print("Partition Result:")
print("SGX Layers:", sgx_layers)
print("GPU Layers:", gpu_layers)

Grid Search Progress: 100%|██████████| 810000/810000 [00:52<00:00, 15445.40it/s]

Partition Result:
SGX Layers: [2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 19, 20, 22, 24, 25, 26, 29, 32, 35, 38, 41, 44, 54, 57]
GPU Layers: [1, 4, 11, 15, 17, 18, 21, 23, 27, 28, 30, 31, 33, 34, 36, 37, 39, 40, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 58, 59, 60]
